# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f404cd17340>
├── 'a' --> tensor([[ 0.5416, -0.1684, -0.6727],
│                   [ 1.1070, -0.7641,  0.3552]])
└── 'x' --> <FastTreeValue 0x7f404cd7ad00>
    └── 'c' --> tensor([[-1.0623, -0.2272, -1.0072,  1.0367],
                        [-1.1642, -0.6360, -1.9730,  0.7063],
                        [ 0.2449,  1.3592, -0.4793, -0.6210]])

In [4]:
t.a

tensor([[ 0.5416, -0.1684, -0.6727],
        [ 1.1070, -0.7641,  0.3552]])

In [5]:
%timeit t.a

78.6 ns ± 1.2 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f404cd17340>
├── 'a' --> tensor([[-0.6061, -0.5670, -0.7513],
│                   [ 1.6048,  0.0759, -0.2600]])
└── 'x' --> <FastTreeValue 0x7f404cd7ad00>
    └── 'c' --> tensor([[-1.0623, -0.2272, -1.0072,  1.0367],
                        [-1.1642, -0.6360, -1.9730,  0.7063],
                        [ 0.2449,  1.3592, -0.4793, -0.6210]])

In [7]:
%timeit t.a = new_value

83.9 ns ± 0.883 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5416, -0.1684, -0.6727],
               [ 1.1070, -0.7641,  0.3552]]),
    x: Batch(
           c: tensor([[-1.0623, -0.2272, -1.0072,  1.0367],
                      [-1.1642, -0.6360, -1.9730,  0.7063],
                      [ 0.2449,  1.3592, -0.4793, -0.6210]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5416, -0.1684, -0.6727],
        [ 1.1070, -0.7641,  0.3552]])

In [11]:
%timeit b.a

76.4 ns ± 0.666 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2779,  0.2451,  2.0551],
               [ 0.5009, -1.5996,  0.1360]]),
    x: Batch(
           c: tensor([[-1.0623, -0.2272, -1.0072,  1.0367],
                      [-1.1642, -0.6360, -1.9730,  0.7063],
                      [ 0.2449,  1.3592, -0.4793, -0.6210]]),
       ),
)

In [13]:
%timeit b.a = new_value

663 ns ± 5.89 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.06 µs ± 10.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.1 µs ± 95.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

285 µs ± 7.54 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

275 µs ± 5.81 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f404cd121f0>
├── 'a' --> tensor([[[ 0.5416, -0.1684, -0.6727],
│                    [ 1.1070, -0.7641,  0.3552]],
│           
│                   [[ 0.5416, -0.1684, -0.6727],
│                    [ 1.1070, -0.7641,  0.3552]],
│           
│                   [[ 0.5416, -0.1684, -0.6727],
│                    [ 1.1070, -0.7641,  0.3552]],
│           
│                   [[ 0.5416, -0.1684, -0.6727],
│                    [ 1.1070, -0.7641,  0.3552]],
│           
│                   [[ 0.5416, -0.1684, -0.6727],
│                    [ 1.1070, -0.7641,  0.3552]],
│           
│                   [[ 0.5416, -0.1684, -0.6727],
│                    [ 1.1070, -0.7641,  0.3552]],
│           
│                   [[ 0.5416, -0.1684, -0.6727],
│                    [ 1.1070, -0.7641,  0.3552]],
│           
│                   [[ 0.5416, -0.1684, -0.6727],
│                    [ 1.1070, -0.7641,  0.3552]]])
└── 'x' --> <FastTreeValue 0x7f404cd12730>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

48.1 µs ± 812 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3f96fd7f40>
├── 'a' --> tensor([[ 0.5416, -0.1684, -0.6727],
│                   [ 1.1070, -0.7641,  0.3552],
│                   [ 0.5416, -0.1684, -0.6727],
│                   [ 1.1070, -0.7641,  0.3552],
│                   [ 0.5416, -0.1684, -0.6727],
│                   [ 1.1070, -0.7641,  0.3552],
│                   [ 0.5416, -0.1684, -0.6727],
│                   [ 1.1070, -0.7641,  0.3552],
│                   [ 0.5416, -0.1684, -0.6727],
│                   [ 1.1070, -0.7641,  0.3552],
│                   [ 0.5416, -0.1684, -0.6727],
│                   [ 1.1070, -0.7641,  0.3552],
│                   [ 0.5416, -0.1684, -0.6727],
│                   [ 1.1070, -0.7641,  0.3552],
│                   [ 0.5416, -0.1684, -0.6727],
│                   [ 1.1070, -0.7641,  0.3552]])
└── 'x' --> <FastTreeValue 0x7f3f96fb0c40>
    └── 'c' --> tensor([[-1.0623, -0.2272, -1.0072,  1.0367],
                        [-1.1642, -0.6360, -1.9730,  0.7063],
                 

In [23]:
%timeit t_cat(trees)

43.3 µs ± 578 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

82.2 µs ± 896 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.0623, -0.2272, -1.0072,  1.0367],
                       [-1.1642, -0.6360, -1.9730,  0.7063],
                       [ 0.2449,  1.3592, -0.4793, -0.6210]],
              
                      [[-1.0623, -0.2272, -1.0072,  1.0367],
                       [-1.1642, -0.6360, -1.9730,  0.7063],
                       [ 0.2449,  1.3592, -0.4793, -0.6210]],
              
                      [[-1.0623, -0.2272, -1.0072,  1.0367],
                       [-1.1642, -0.6360, -1.9730,  0.7063],
                       [ 0.2449,  1.3592, -0.4793, -0.6210]],
              
                      [[-1.0623, -0.2272, -1.0072,  1.0367],
                       [-1.1642, -0.6360, -1.9730,  0.7063],
                       [ 0.2449,  1.3592, -0.4793, -0.6210]],
              
                      [[-1.0623, -0.2272, -1.0072,  1.0367],
                       [-1.1642, -0.6360, -1.9730,  0.7063],
                       [ 0.2449,  1.3592, -0.4793, -0.6210]],

In [26]:
%timeit Batch.stack(batches)

110 µs ± 2.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.0623, -0.2272, -1.0072,  1.0367],
                      [-1.1642, -0.6360, -1.9730,  0.7063],
                      [ 0.2449,  1.3592, -0.4793, -0.6210],
                      [-1.0623, -0.2272, -1.0072,  1.0367],
                      [-1.1642, -0.6360, -1.9730,  0.7063],
                      [ 0.2449,  1.3592, -0.4793, -0.6210],
                      [-1.0623, -0.2272, -1.0072,  1.0367],
                      [-1.1642, -0.6360, -1.9730,  0.7063],
                      [ 0.2449,  1.3592, -0.4793, -0.6210],
                      [-1.0623, -0.2272, -1.0072,  1.0367],
                      [-1.1642, -0.6360, -1.9730,  0.7063],
                      [ 0.2449,  1.3592, -0.4793, -0.6210],
                      [-1.0623, -0.2272, -1.0072,  1.0367],
                      [-1.1642, -0.6360, -1.9730,  0.7063],
                      [ 0.2449,  1.3592, -0.4793, -0.6210],
                      [-1.0623, -0.2272, -1.0072,  1.0367],
                   

In [28]:
%timeit Batch.cat(batches)

193 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

733 µs ± 40.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
